# Federated Isolation Forest  with Flex  for Anomaly Detection

In [ ]:
from flexanomalies.utils import IsolationForest
from flexanomalies.utils.load_data import load_and_split_dot_mat, federate_data
from flexanomalies.pool.primitives_iforest import (
    build_server_model_if,
    copy_model_to_clients_if,
    train_if,
    aggregate_if,
    set_aggregated_weights_if,
    get_clients_weights_if,
)
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool

In [2]:
model_params = {"n_estimators": 100, "contamination": 0.1}

In [3]:
X_train, X_test, y_train, y_test = load_and_split_dot_mat(
    "flexanomalies/datasets/data/shuttle.mat", 0.3
)

In [4]:
model = IsolationForest(**model_params)

In [5]:
flex_dataset = federate_data(5, X_train, y_train)

In [6]:
pool = FlexPool.client_server_pool(
    fed_dataset=flex_dataset,
    server_id="iforest_server",
    init_func=build_server_model_if,
    model=model,
)

In [7]:
pool.servers.map(copy_model_to_clients_if, pool.clients)
pool.clients.map(train_if)
pool.aggregators.map(get_clients_weights_if, pool.clients)
pool.aggregators.map(aggregate_if)
pool.aggregators.map(set_aggregated_weights_if, pool.servers)
output_model = pool.servers._models["iforest_server"]["model"]
output_model.evaluate(X_test, y_test)

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.
Acc: 96.796% 

Precision: 0.688 

F1score: 0.811 

Recall: 0.988 

AUC_ROC: 0.977 



In [9]:
output_model.result_metrics_

{'Accuracy': '96.796%',
 'Precision': '0.688',
 'F1': '0.811',
 'Recall': '0.988',
 'AUC_ROC': '0.977'}

In [8]:
save_experiments_results(
    "Iforest",
    output_model,
    "test_iforest_notebook",
    model_params,
    "shuttle.mat",
    5,
    3,
    0.3,
)